In [1]:
# Import the dependencies
from sqlalchemy import create_engine, text, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
# Create a base class for declarating class definitions to produce Table objects
Base = declarative_base()

In [2]:
# Define our pet table
class Pet(Base):
    __tablename__ = 'pet'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    age = Column(Integer)

In [3]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

FacadeDict({'pet': Table('pet', MetaData(), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(), table=<pet>), Column('type', String(), table=<pet>), Column('age', Integer(), table=<pet>), schema=None)})

In [4]:
# Create our database engine
engine = create_engine('sqlite:///pets.sqlite')

In [5]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [6]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

## Create Data

In [7]:
# Note that adding to the session does not update the table. It queues up those queries.
session.add(Pet(name='Justin Timbersnake', type='snek', age=2))
session.add(Pet(name='Pawtrick Stewart', type='good boy', age=10))
session.add(Pet(name='Godzilla', type='iguana', age=1))
session.add(Pet(name='Marshmallow', type='polar bear', age=4))

In [8]:
# The data hasn't been added yet
engine.execute(text('select * from pet')).fetchall()

[]

In [9]:
# We can use the new attribute to see the queue of data ready to go into the database
session.new

IdentitySet([<__main__.Pet object at 0x7fc810a42560>, <__main__.Pet object at 0x7fc810a43fa0>, <__main__.Pet object at 0x7fc810a43d30>, <__main__.Pet object at 0x7fc810a42fe0>])

In [10]:
# Commit() flushes whatever remaining changes remain to the database, and commits the transaction.
session.commit()

In [11]:
# Nothing new to add
session.new

IdentitySet([])

In [12]:
# Query the database
session.query(Pet.name, Pet.type, Pet.age).all()

[('Justin Timbersnake', 'snek', 2),
 ('Pawtrick Stewart', 'good boy', 10),
 ('Godzilla', 'iguana', 1),
 ('Marshmallow', 'polar bear', 4)]

## Update Data

In [16]:
pet = session.query(Pet).filter_by(name="Marshmallow")
pet.age

AttributeError: 'Query' object has no attribute 'age'

In [20]:
# Create a query and then run update on it
pet = session.query(Pet).filter_by(name="Marshmallow").first()
print(f"Before: {pet.name} {pet.type} {pet.age}")

pet.age = pet.age + 1
print(f"After: {pet.name} {pet.type} {pet.age}")

Before: Marshmallow polar bear 4
After: Marshmallow polar bear 5


In [21]:
# For modifications, we can use the dirty attribute
session.dirty

IdentitySet([<__main__.Pet object at 0x7fc810a42fe0>])

In [22]:
# Commit Transaction
session.commit()

In [23]:
# Session is up-to-date
session.dirty

IdentitySet([])

In [24]:
# Query the database.
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Marshmallow', 'polar bear', 5)]

## Delete Data

In [26]:
# Create a query and then delete the row collected
pet = session.query(Pet).filter_by(id=4).one()
pet.name

'Marshmallow'

In [27]:
session.delete(pet)

In [28]:
# The deletion queue
session.deleted

IdentitySet([<__main__.Pet object at 0x7fc810a42fe0>])

In [29]:
session.commit()

In [30]:
session.deleted

IdentitySet([])

In [31]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1)]

In [32]:
# Close the session
session.close()